# Quickstart
## Defining the orbit: Orbit objects
The core of poliastro are the Orbit objects inside the *poliastro.twobody* module. They store all the required information to define an orbit:
* The body acting as the central body of the orbit, for example the Earth.
* The position and velocity vectors or the orbital elements.
* The time at which the orbit is defined.

First of all, we have to import the relevant modules and classes:

In [1]:
%matplotlib inline

from astropy import units as u

from poliastro.bodies import Earth, Mars, Sun
from poliastro.twobody import Orbit

## From position and velocity

There are several methods available to create *Orbit* objects. For example, if we have the position and velocity vectors we can use *from_vectors()*:

In [2]:
# Data from Curtis, example 4.3
r = [-6045, -3490, 2500] * u.km
v = [-3.457, 6.618, 2.533] * u.km / u.s

orb = Orbit.from_vectors(Earth, r, v)

And that’s it! Notice a couple of things:

* Defining vectorial physical quantities using Astropy units is very easy. The list is automatically converted to a *astropy.units.Quantity*, which is actually a subclass of NumPy arrays.

* If we display the orbit we just created, we get a string with the radius of pericenter, radius of apocenter, inclination, reference frame and attractor:

In [3]:
orb

7283 x 10293 km x 153.2 deg (GCRS) orbit around Earth (♁) at epoch J2000.000 (TT)

* If no time is specified, then a default value is assigned:

In [4]:
orb.epoch

<Time object: scale='tt' format='jyear_str' value=J2000.000>

In [5]:
orb.epoch.iso

'2000-01-01 12:00:00.000'

* The reference frame of the orbit will be one pseudo-inertial frame around the attractor. You can retrieve it using the **frame** property:

In [6]:
orb.get_frame()

<GCRS Frame (obstime=J2000.000, obsgeoloc=(0., 0., 0.) m, obsgeovel=(0., 0., 0.) m / s)>

## Intermezzo: quick visualization of the orbit
If we’re working on interactive mode (for example, using the wonderful Jupyter notebook) we can immediately plot the current orbit:

In [7]:
orb.plot()

ValueError: subformat not allowed for format jyear_str

This plot is made in the so called *perifocal* frame, which means:
* we’re visualizing the plane of the orbit itself,
* the (𝑥) axis points to the pericenter, and
* the (𝑦) axis is turned 90∘ in the direction of the orbit.

The dotted line represents the *osculating orbit*: the instantaneous Keplerian orbit at that point. This is relevant in the context of perturbations, when the object shall deviate from its Keplerian orbit.

## From classical orbital elements
We can also define an *Orbit* using a set of six parameters called orbital elements. Although there are several of these element sets, each one with its advantages and drawbacks, right now poliastro supports the classical orbital elements:
* Semimajor axis (𝑎)
* Eccentricity (𝑒)
* Inclination (𝑖)
* Right ascension of the ascending node (Ω)
* Argument of pericenter (𝜔)
* True anomaly (𝜈)

In this case, we’d use the method *from_classical()*:

In [8]:
# Data for Mars at J2000 from JPL HORIZONS
a = 1.523679 * u.AU
ecc = 0.093315 * u.one
inc = 1.85 * u.deg
raan = 49.562 * u.deg
argp = 286.537 * u.deg
nu = 23.33 * u.deg

orb = Orbit.from_classical(Sun, a, ecc, inc, raan, argp, nu)

Notice that whether we create an *Orbit* from (𝑟) and (𝑣) or from elements we can access many mathematical properties of the orbit:

In [9]:
orb.period.to(u.day)

<Quantity 686.97138348 d>

In [10]:
orb.v

<Quantity [ 1.16420212, 26.29603633,  0.52229379] km / s>

To see a complete list of properties, check out the *poliastro.twobody.orbit.Orbit* class on the API reference.

## Moving forward in time: propagation

Now that we have defined an orbit, we might be interested in computing how is it going to evolve in the future. In the context of orbital mechanics, this process is known as propagation, and can be performed with the propagate method of Orbit objects:

In [11]:
from poliastro.examples import iss

iss

6772 x 6790 km x 51.6 deg (GCRS) orbit around Earth (♁) at epoch 2013-03-18 12:00:00.000 (UTC)

In [12]:
iss.epoch

<Time object: scale='utc' format='iso' value=2013-03-18 12:00:00.000>

In [13]:
iss.nu.to(u.deg)

<Quantity 46.59580468 deg>

In [14]:
iss.n.to(u.deg / u.min)

<Quantity 3.88701058 deg / min>

Using the *propagate()* method we can now retrieve the position of the ISS after some time:

In [15]:
iss_30m = iss.propagate(30 * u.min)
iss_30m.epoch  # Notice we advanced the epoch!

<Time object: scale='utc' format='iso' value=2013-03-18 12:30:00.000>

In [16]:
iss_30m.nu.to(u.deg)

<Quantity 163.1409362 deg>

For more advanced propagation options, check out the *poliastro.twobody.propagation* module.

## Studying non-keplerian orbits: perturbations

Apart from the Keplerian propagators, poliastro also allows the user to define custom perturbation accelerations to study non Keplerian orbits, thanks to Cowell’s method:

In [17]:
from numba import njit
import numpy as np

from poliastro.core.propagation import func_twobody
from poliastro.twobody.propagation import cowell


r0 = [-2384.46, 5729.01, 3050.46] * u.km
v0 = [-7.36138, -2.98997, 1.64354] * u.km / u.s
initial = Orbit.from_vectors(Earth, r0, v0)

@njit
def accel(t0, state, k):
    v_vec = state[3:]
    norm_v = (v_vec * v_vec).sum() ** 0.5
    return 1e-5 * v_vec / norm_v

def f(t0, u_, k):
    du_kep = func_twobody(t0, u_, k)
    ax, ay, az = accel(t0, u_, k)
    du_ad = np.array([0, 0, 0, ax, ay, az])
    return du_kep + du_ad

initial.propagate(3 * u.day, method=cowell, f=f)

18255 x 21848 km x 28.0 deg (GCRS) orbit around Earth (♁) at epoch J2000.008 (TT)

Some natural perturbations are available in poliastro to be used directly in this way. For instance, let us examine the effect of J2 perturbation:

In [18]:
from poliastro.core.perturbations import J2_perturbation

tofs = [48.0] * u.h

def f(t0, u_, k):
    du_kep = func_twobody(t0, u_, k)
    ax, ay, az = J2_perturbation(
        t0, u_, k, J2=Earth.J2.value, R=Earth.R.to(u.km).value
    )
    du_ad = np.array([0, 0, 0, ax, ay, az])
    return du_kep + du_ad

final = initial.propagate(tofs, method=cowell, f=f)

The J2 perturbation changes the orbit parameters (from Curtis example 12.2):

In [19]:
((final.raan - initial.raan) / tofs).to(u.deg / u.h)

<Quantity [-0.17232668] deg / h>

In [21]:
((final.argp - initial.argp) / tofs).to(u.deg / u.h)

<Quantity [0.28220397] deg / h>

For more available perturbation options, see the *poliastro.twobody.perturbations* module.

## Studying artificial perturbations: thrust

In addition to natural perturbations, poliastro also has built-in artificial perturbations (thrusts) aimed at intentional change of some orbital elements. Let us simultaneously change eccentricy and inclination:

In [24]:
from poliastro.twobody.thrust import change_inc_ecc
from poliastro.twobody import Orbit
from poliastro.bodies import Earth
from poliastro.twobody.propagation import cowell
from astropy import units as u
from astropy.time import Time

ecc_0, ecc_f = 0.4, 0.0
a = 42164  # km
inc_0 = 0.0  # rad, baseline
inc_f = (20.0 * u.deg).to(u.rad).value  # rad
argp = 0.0  # rad, the method is efficient for 0 and 180
f = 2.4e-7  # km / s2

# Retrieve r and v from initial orbit
s0 = Orbit.from_classical(
    Earth,
    a * u.km,
    ecc_0 * u.one,
    inc_0 * u.deg,
    0 * u.deg,
    argp * u.deg,
    0 * u.deg,
)
a_d, _, _, t_f = change_inc_ecc(s0, ecc_f, inc_f, f)

# Propagate orbit
def f_geo(t0, u_, k):
    du_kep = func_twobody(t0, u_, k)
    ax, ay, az = a_d(t0, u_, k)
    du_ad = np.array([0, 0, 0, ax, ay, az])
    return du_kep + du_ad

sf = s0.propagate(t_f * u.s, method=cowell, f=f_geo, rtol=1e-8)

The thrust changes orbit parameters as desired (within errors):

In [26]:
sf.inc, sf.ecc

(<Quantity 0.34719735 rad>, <Quantity 0.00894521>)

For more available perturbation options, see the *poliastro.twobody.thrust* module.

## Changing the orbit: Maneuver objects

poliastro helps us define several in-plane and general out-of-plane maneuvers with the Maneuver class inside the *poliastro.maneuver* module.

Each Maneuver consists on a list of impulses Δ𝑣𝑖 (changes in velocity) each one applied at a certain instant 𝑡𝑖. The simplest maneuver is a single change of velocity without delay: you can recreate it either using the impulse() method or instantiating it directly.

In [28]:
from poliastro.maneuver import Maneuver

dv = [5, 0, 0] * u.m / u.s

man = Maneuver.impulse(dv)
man = Maneuver((0 * u.s, dv))  # Equivalent

There are other useful methods you can use to compute common in-plane maneuvers, notably *hohmann()* and *bielliptic()* for Hohmann and bielliptic transfers respectively. Both return the corresponding Maneuver object, which in turn you can use to calculate the total cost in terms of velocity change ∑|Δ𝑣𝑖| and the transfer time:

In [31]:
ss_i = Orbit.circular(Earth, alt=700 * u.km)
ss_i

7078 x 7078 km x 0.0 deg (GCRS) orbit around Earth (♁) at epoch J2000.000 (TT)

In [33]:
hoh = Maneuver.hohmann(ss_i, 36000 * u.km)
hoh.get_total_cost()

<Quantity 3.6173999 km / s>

In [35]:
hoh.get_total_time()

<Quantity 15729.73314712 s>

You can also retrieve the individual vectorial impulses:

In [37]:
hoh.impulses[0]

(<Quantity 0. s>, <Quantity [   0.       , 2197.3992373,    0.       ] m / s>)

In [39]:
hoh[0]

(<Quantity 0. s>, <Quantity [   0.       , 2197.3992373,    0.       ] m / s>)

In [41]:
tuple(val.decompose([u.km, u.s]) for val in hoh[1])

(<Quantity 15729.73314712 s>,
 <Quantity [ 0.        , -1.42000067,  0.        ] km / s>)

To actually retrieve the resulting Orbit after performing a maneuver, use the method *apply_maneuver()*:

In [43]:
ss_f = ss_i.apply_maneuver(hoh)
ss_f

36000 x 36000 km x 0.0 deg (GCRS) orbit around Earth (♁) at epoch J2000.000 (TT)

## More advanced plotting: OrbitPlotter objects

We previously saw the poliastro.plotting.plot() function to easily plot orbits. Now we’d like to plot several orbits in one graph (for example, the maneuver we computed in the previous section). For this purpose, we have OrbitPlotter objects in the plotting module.

These objects hold the perifocal plane of the first Orbit we plot in them, projecting any further trajectories on this plane. This allows to easily visualize in two dimensions:

In [45]:
from poliastro.plotting import OrbitPlotter2D

op = OrbitPlotter2D()
ss_a, ss_f = ss_i.apply_maneuver(hoh, intermediate=True)
op.plot(ss_i, label="Initial orbit")
op.plot(ss_a, label="Transfer orbit")
op.plot(ss_f, label="Final orbit")

## Where are the planets? Computing ephemerides

New in version 0.14.0.

Thanks to Astropy and jplephem, poliastro can read Satellite Planet Kernel (SPK) files, part of NASA’s SPICE toolkit. This means that we can query the position and velocity of the planets of the Solar System.

The poliastro.ephem.Ephem class allows us to retrieve a planetary orbit using low precision ephemerides available in Astropy and an astropy.time.Time:

In [47]:
from astropy import time
epoch = time.Time("2020-04-29 10:43")  # UTC by default

And finally, retrieve the planet ephemerides:

In [49]:
from poliastro.ephem import Ephem

earth = Ephem.from_body(Earth, epoch.tdb)
earth

Ephemerides at 1 epochs from 2020-04-29 10:44:09.186 (TDB) to 2020-04-29 10:44:09.186 (TDB)

In [53]:
from astropy.coordinates import solar_system_ephemeris
solar_system_ephemeris.set("jpl")

<ScienceState solar_system_ephemeris: 'jpl'>

If we want to retrieve the osculating orbit at a given epoch, we can do so using from_ephem():

In [54]:
Orbit.from_ephem(Sun, earth, epoch)

1 x 1 AU x 23.4 deg (HCRS) orbit around Sun (☉) at epoch 2020-04-29 10:43:00.000 (UTC)

## Traveling through space: solving the Lambert problem

The determination of an orbit given two position vectors and the time of flight is known in celestial mechanics as Lambert’s problem, also known as two point boundary value problem. This contrasts with Kepler’s problem or propagation, which is rather an initial value problem.

The package poliastro.iod holds the different raw algorithms to solve Lambert’s problem, provided the main attractor’s gravitational constant, the two position vectors and the time of flight. As you can imagine, being able to compute the positions of the planets as we saw in the previous section is the perfect complement to this feature!

Since poliastro version 0.13 it is possible to solve Lambert’s problem by making use of the poliastro.maneuver module. We just need to pass as arguments the initial and final orbits and the output will be a poliastro.maneuver.Maneuver instance. Time of flight is computed internally since orbits epochs are known.

For instance, this is a simplified version of the example “Going to Mars with Python using poliastro”, where the orbit of the Mars Science Laboratory mission (rover Curiosity) is determined:

In [56]:
date_launch = time.Time('2011-11-26 15:02', scale='utc')
date_arrival = time.Time('2012-08-06 05:17', scale='utc')

ss0 = Orbit.from_ephem(Sun, Ephem.from_body(Earth, date_launch), date_launch)
ssf = Orbit.from_ephem(Sun, Ephem.from_body(Mars, date_arrival), date_arrival)

man_lambert = Maneuver.lambert(ss0, ssf)
dv_a, dv_b = man_lambert.impulses

<ipython-input-56-9cc1a70d010c>:4: TimeScaleWarning:

Input time was converted to scale='tdb' with value ['2011-11-26 15:03:06.183']. Use Time(..., scale='tdb') instead.

<ipython-input-56-9cc1a70d010c>:5: TimeScaleWarning:

Input time was converted to scale='tdb' with value ['2012-08-06 05:18:07.183']. Use Time(..., scale='tdb') instead.



In [58]:
dv_a

(<Quantity 0. s>,
 <Quantity [-2064.35606458,  2587.62245415,   239.60146757] m / s>)

In [60]:
dv_b

(<Quantity 21910501. s>,
 <Quantity [ 3332.19170739,   680.5317439 , -1090.15094977] m / s>)

## Fetching Orbits from external sources

As of now, poliastro supports fetching orbits from 2 online databases from Jet Propulsion Laboratory: SBDB and Horizons.

JPL Horizons can be used to generate ephemerides for solar-system bodies, while JPL SBDB (Small-Body Database Browser) provides model orbits for all known asteroids and many comets.

The data is fetched using the wrappers to these services provided by astroquery.

In [73]:
epoch = time.Time("2020-04-29 10:43")
Ephem.from_horizons("Ceres", epoch)
Orbit.from_sbdb("Apophis")

1 x 1 AU x 3.3 deg (HeliocentricEclipticIAU76) orbit around Sun (☉) at epoch 2459200.500800735 (TDB)

## Creating a CZML document

We can create CZML documents which can then be visualized with the help of Cesium. First we load the orbital data and the CZML Extractor:

In [75]:
from poliastro.examples import molniya, iss
from poliastro.czml.extract_czml import CZMLExtractor

Then we specify the starting and ending epoch, as well as the number of sample points (the higher the number, the more accurate the trajectory):

In [78]:
start_epoch = iss.epoch
end_epoch = iss.epoch + molniya.period
sample_points = 10

extractor = CZMLExtractor(start_epoch, end_epoch, sample_points)

extractor.add_orbit(molniya, label_text="Molniya")
extractor.add_orbit(iss, label_text="ISS")